Database ve tabloların oluşturulup kayıtların eklenmesi

1 - Find the employees who get paid more than Rodney Weaver.

2 - Find the average, min and max salaries

3 - Find the employees whose salary is more than 8700. Our query should return first name, last name, and salary info of the employees.

4 - Find the employees (first name, last name from employees table) who work under the Operations department (departments table). Our query should return first name and last name info.

5 - Find the employees (first name, last name from employees table) who work under the Technology department (departments table). Our query should return first name and last name info.

6 - Find the average salary of female employees.

7 - Find the average salaries of each department.

8 - Find the oldest and newest employees.

9 - Find the hiring date and department of the highest paid employee

10 - Find the hiring date and department of the lowest paid employee